In [218]:
import pandas as pd 
import numpy as np
import matplotlib.pyplot as plt

In [219]:
df = pd.read_csv('./_data.csv', sep=',')
includes = []
for i in range(len(df)):
    if 'Москва' not in df['Адрес'][i]:
        includes.append(i)
print(df.columns.to_list())
df_clean=df.drop(includes)
print(len(df_clean))
df = df_clean.copy().drop(['Тип', 'Unnamed: 0', 'Телефоны', 'Серия дома', 'Название ЖК', 'Ссылка на объявление','Описание','Адрес','Дополнительно'], axis=1)
df['Ремонт'].fillna('неизвестно')

df.to_csv('data.csv', sep='\t')

['Unnamed: 0', 'ID  объявления', 'Количество комнат', 'Тип', 'Метро', 'Адрес', 'Площадь, м2', 'Дом', 'Парковка', 'Цена', 'Телефоны', 'Описание', 'Ремонт', 'Площадь комнат, м2', 'Балкон', 'Окна', 'Санузел', 'Можно с детьми/животными', 'Дополнительно', 'Название ЖК', 'Серия дома', 'Высота потолков, м', 'Лифт', 'Мусоропровод', 'Ссылка на объявление']
19737


In [220]:
df.head(2) 


,ID объявления,Количество комнат,Метро,"Площадь, м2",Дом,Парковка,Цена,Ремонт,"Площадь комнат, м2",Балкон,Окна,Санузел,Можно с детьми/животными,"Высота потолков, м",Лифт,Мусоропровод
0,271271157,4,м. Смоленская (9 мин пешком),200.0/20.0,"5/16, Монолитный",подземная,"500000.0 руб./ За месяц, Залог - 500000 руб., ...",Дизайнерский,NaN,NaN,NaN,NaN,"Можно с детьми, Можно с животными",3.0,"Пасс (4), Груз (1)",Да
1,271634126,4,м. Смоленская (8 мин пешком),198.0/95.0/18.0,"5/16, Монолитно-кирпичный",подземная,"500000.0 руб./ За месяц, Залог - 500000 руб., ...",Дизайнерский,25 25 20 25,NaN,На улицу и двор,"Совмещенный (2), Раздельный (1)",Можно с детьми,3.5,"Пасс (1), Груз (1)",Нет


In [221]:
#Приведим все комнаты в числовое или около того значение
df['Количество комнат'] = df['Количество комнат'].str[0]


In [222]:
#Убираем NaN в кол-ве комнат
df['Количество комнат'].fillna(int(df['Количество комнат'].mode()), inplace=True)

C:\Users\User\AppData\Local\Temp\ipykernel_12664\1043790658.py:2: FutureWarning: Calling int on a single element Series is deprecated and will raise a TypeError in the future. Use int(ser.iloc[0]) instead
  df['Количество комнат'].fillna(int(df['Количество комнат'].mode()), inplace=True)


In [223]:
# Считаем, что отдельные площади кухни и комнат нам не важны, и оставляет только общую площадь.
df['Площадь кв, м2'] = [row.split('/')[0] for row in df['Площадь, м2']]
# И убираем две колонки с площадью
# Удаляем колонки со старой площадью
df = df.drop(['Площадь, м2'], axis = 1)
df = df.drop(['Площадь комнат, м2'], axis = 1)

In [224]:
# Убираем NaN в колонке 'Окна'
df['Окна'] = df['Окна'].fillna('Во двор')

In [225]:
# Убираем NaN в колонке 'Ремонт'
df['Ремонт'] = df['Ремонт'].fillna('Косметический')

In [226]:
# Вытаскиваем информацию из колонки 'дом'
df['Тип дома'] = [row.split(', ')[-1] if len(row.split(', ')[-1]) > 6 else 'Неизвестно' for row in df['Дом']]
df['Этажность дома'] = [int(row.split('/')[1].split(',')[0]) for row in df['Дом']]
df['Этаж квартиры'] = [int(row.split('/')[0]) for row in df['Дом']]
df = df.drop(['Дом'], axis = 1)

In [227]:
# Избавляемся от NaN для лифтов, если меньше 5 этажей - нет лифта, в остальном по одному каждого лифта
df['Лифт'] = [row if row == row else
              ['Пасс (1), Груз (1)' if df['Этажность дома'].iloc[i]>5 else 'Пасс (0), Груз (0)'][0]
              for i, row in enumerate(df['Лифт'])]


In [228]:
# Убираем NaN в колонке 'Мусоропровод'
df['Мусоропровод'] = df['Мусоропровод'].fillna('Нет')

In [229]:
# Убираем NaN в колонке 'Высота потолков, м'
df['Высота потолков, м'] = df['Высота потолков, м'].fillna(round(df['Высота потолков, м'].mean(), 2))


In [230]:
# Убираем NaN в колонке 'Санузел'
df['Санузел'] = df['Санузел'].fillna('Совмещенный (1)')
df['Санузел совмещенный'] = [row.split(' ')[1] if (row.split(' ')[0]) == 'Совмещенный' else '(0)' for row in df['Санузел']]
df['Санузел раздельный'] = [row.split(' ')[-1] if (row.split(' ')[-2]) == 'Раздельный' else '(0)' for row in df['Санузел']]
df['Санузел совмещенный'] = [row.split('(')[1] for row in df['Санузел совмещенный']]
df['Санузел совмещенный'] = [int(row.split(')')[0]) for row in df['Санузел совмещенный']]
df['Санузел раздельный'] = [row.split('(')[1] for row in df['Санузел раздельный']]
df['Санузел раздельный'] = [int(row.split(')')[0]) for row in df['Санузел раздельный']]
df = df.drop(['Санузел'], axis = 1)

In [231]:
# Убираем NaN в колонке 'Балкон'
df['Балкон'] = df['Балкон'].fillna('Балкон (0), Лоджия (0)')
df['Лоджия'] = [row.split(' ')[-1] if (row.split(' ')[-2]) == 'Лоджия' else '(0)' for row in df['Балкон']]
df['Балкон'] = [row.split(' ')[1] if row.split(' ')[0] == 'Балкон' else '(0)' for row in df['Балкон']]
df['Лоджия'] = [row.split('(')[1] for row in df['Лоджия']]
df['Лоджия'] = [int(row.split(')')[0]) for row in df['Лоджия']]
df['Балкон'] = [row.split('(')[1] for row in df['Балкон']]
df['Балкон'] = [int(row.split(')')[0]) for row in df['Балкон']]

In [232]:
df['Парковка'] = df['Парковка'].fillna('наземная')

In [233]:
df['Можно с детьми/животными']=df['Можно с детьми/животными'].fillna('по договорённости')

In [234]:
#Переводим расстояние до метро в метры в зависимости от указанного способо передвижения, для NaN задаём медиану
df['время до метро'] = df['Метро'].str.replace(')', '').str.split('(').str[-1]
df[['время в минутах', 'способ']] = df['время до метро'].str.extract(r'(\d+) мин\s(пешком|на машине)')
df['способ'] = df['способ'].apply(lambda x: 70 if x == 'пешком' else 750)
df['способ'] = df['способ'].fillna(0)
df['время в минутах'] = df['время в минутах'].fillna(0)
df['distance to subway, m'] = df['способ'] * df['время в минутах'].astype(int)
median_value = df['distance to subway, m'].median()
df['distance to subway, m'] = df['distance to subway, m'].replace(0, median_value)


In [235]:
df.rename(columns={'ID  объявления': 'Ad_ID'}, inplace=True)
df.rename(columns={'Количество комнат': 'Number_of_Rooms'}, inplace=True)
df.rename(columns={'Метро': 'Metro'}, inplace=True)
df.rename(columns={'Адрес': 'Address'}, inplace=True)
df.rename(columns={'Парковка': 'Parking'}, inplace=True)
df.rename(columns={'Цена': 'Price'}, inplace=True)
df.rename(columns={'Ремонт': 'Renovation'}, inplace=True)
df.rename(columns={'Балкон': 'Balcony'}, inplace=True)
df.rename(columns={'Окна': 'Windows'}, inplace=True)
df.rename(columns={'Можно с детьми/животными': 'Children/Pets_Allowed'}, inplace=True)
df.rename(columns={'Дополнительно': 'Additional Info'}, inplace=True)
df.rename(columns={'Высота потолков, м': 'Ceiling_Height_M'}, inplace=True)
df.rename(columns={'Лифт': 'Elevator'}, inplace=True)
df.rename(columns={'Мусоропровод': 'Garbage_Chute'}, inplace=True)
df.rename(columns={'Площадь кв, м2': 'Area_sq_m'}, inplace=True)
df.rename(columns={'Тип дома': 'House_Type'}, inplace=True)
df.rename(columns={'Этажность дома': 'Number_of_Floors'}, inplace=True)
df.rename(columns={'Этаж квартиры': 'Floor_of_Apartment'}, inplace=True)
df.rename(columns={'Санузел совмещенный': 'Combined_Bathroom'}, inplace=True)
df.rename(columns={'Санузел раздельный': 'Separate_Bathroom'}, inplace=True)
df.rename(columns={'Лоджия': 'Balconette'}, inplace=True)
df.drop(['время до метро'], axis=1, inplace = True)
df.drop(['время в минутах'], axis=1, inplace = True)
df.drop(['способ'], axis=1, inplace = True)
df.drop(['Metro'], axis=1, inplace = True)

In [236]:
df['Parking'] = df['Parking'].apply(lambda x: x.capitalize())
df.head(10) 

,Ad_ID,Number_of_Rooms,Parking,Price,Renovation,Balcony,Windows,Children/Pets_Allowed,Ceiling_Height_M,Elevator,Garbage_Chute,Area_sq_m,House_Type,Number_of_Floors,Floor_of_Apartment,Combined_Bathroom,Separate_Bathroom,Balconette,"distance to subway, m"
0,271271157,4,Подземная,"500000.0 руб./ За месяц, Залог - 500000 руб., ...",Дизайнерский,0,Во двор,"Можно с детьми, Можно с животными",3.00,"Пасс (4), Груз (1)",Да,200.0,Монолитный,16,5,1,0,0,630
1,271634126,4,Подземная,"500000.0 руб./ За месяц, Залог - 500000 руб., ...",Дизайнерский,0,На улицу и двор,Можно с детьми,3.50,"Пасс (1), Груз (1)",Нет,198.0,Монолитно-кирпичный,16,5,2,1,0,560
2,271173086,4,Подземная,"500000.0 руб./ За месяц, Залог - 500000 руб., ...",Евроремонт,0,На улицу и двор,Можно с детьми,3.20,Пасс (1),Нет,200.0,Неизвестно,16,5,3,0,0,490
3,272197456,4,Подземная,"400000.0 руб./ За месяц, Залог - 400000 руб., ...",Евроремонт,0,На улицу и двор,Можно с животными,3.20,Пасс (1),Нет,170.0,Неизвестно,6,5,3,0,0,210
4,273614615,2,Наземная,"225000.0 руб./ За месяц, Залог - 225000 руб., ...",Евроремонт,0,На улицу и двор,по договорённости,3.90,"Пасс (1), Груз (1)",Да,58.0,Панельный,26,12,2,0,0,490
5,274837728,3,Наземная,"470000.0 руб./ За месяц, Залог - 470000 руб., ...",Дизайнерский,0,Во двор,по договорённости,2.99,"Пасс (1), Груз (1)",Нет,92.0,Неизвестно,7,3,1,0,0,350
6,273643908,5,Наземная,"350000.0 руб./ За месяц, Залог - 350000 руб., ...",Евроремонт,0,На улицу и двор,"Можно с детьми, Можно с животными",3.20,Пасс (1),Нет,213.0,Неизвестно,5,4,2,0,0,280
7,274475342,3,Подземная,"250000.0 руб./ За месяц, Залог - 250000 руб., ...",Евроремонт,0,Во двор,"Можно с детьми, Можно с животными",3.20,Пасс (1),Нет,98.0,Монолитный,4,2,1,1,0,210
8,273973191,3,Открытая,"130000.0 руб./ За месяц, Залог - 130000 руб., ...",Евроремонт,0,На улицу,Можно с животными,3.00,Пасс (1),Нет,120.0,Сталинский,10,5,1,0,0,630
9,272900409,4,Открытая,"210000.0 руб./ За месяц, Залог - 210000 руб., ...",Евроремонт,1,На улицу и двор,Можно с детьми,310.00,Пасс (1),Да,90.0,Сталинский,7,2,1,1,0,700


In [237]:
df.insert(loc=df.columns.to_list().index('Elevator')+1, column='Count_freight_elevator', \
                  value=df['Elevator'].apply(lambda x: int(x[-2]) if 'Груз' in x else 0))
df['Elevator'] = df['Elevator'].apply(lambda x: int(x[x.index('(')+1]) if 'Пасс' in x else 0)
df = df.rename(columns={'Elevator': 'Count_pass_elevator'})

In [238]:
nans = df.isna().any().any()
print(nans)


False


In [239]:
df.head(5)


,Ad_ID,Number_of_Rooms,Parking,Price,Renovation,Balcony,Windows,Children/Pets_Allowed,Ceiling_Height_M,Count_pass_elevator,Count_freight_elevator,Garbage_Chute,Area_sq_m,House_Type,Number_of_Floors,Floor_of_Apartment,Combined_Bathroom,Separate_Bathroom,Balconette,"distance to subway, m"
0,271271157,4,Подземная,"500000.0 руб./ За месяц, Залог - 500000 руб., ...",Дизайнерский,0,Во двор,"Можно с детьми, Можно с животными",3.0,4,1,Да,200.0,Монолитный,16,5,1,0,0,630
1,271634126,4,Подземная,"500000.0 руб./ За месяц, Залог - 500000 руб., ...",Дизайнерский,0,На улицу и двор,Можно с детьми,3.5,1,1,Нет,198.0,Монолитно-кирпичный,16,5,2,1,0,560
2,271173086,4,Подземная,"500000.0 руб./ За месяц, Залог - 500000 руб., ...",Евроремонт,0,На улицу и двор,Можно с детьми,3.2,1,0,Нет,200.0,Неизвестно,16,5,3,0,0,490
3,272197456,4,Подземная,"400000.0 руб./ За месяц, Залог - 400000 руб., ...",Евроремонт,0,На улицу и двор,Можно с животными,3.2,1,0,Нет,170.0,Неизвестно,6,5,3,0,0,210
4,273614615,2,Наземная,"225000.0 руб./ За месяц, Залог - 225000 руб., ...",Евроремонт,0,На улицу и двор,по договорённости,3.9,1,1,Да,58.0,Панельный,26,12,2,0,0,490


In [240]:
#df.to_csv('data.csv', sep=',')

In [241]:
df_encode = df.copy()

In [242]:
# Кодирование понятно чего
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
le.fit(df['Parking'])
df_encode['Parking'] = le.transform(df_encode['Parking'])
df_encode.head(3)



,Ad_ID,Number_of_Rooms,Parking,Price,Renovation,Balcony,Windows,Children/Pets_Allowed,Ceiling_Height_M,Count_pass_elevator,Count_freight_elevator,Garbage_Chute,Area_sq_m,House_Type,Number_of_Floors,Floor_of_Apartment,Combined_Bathroom,Separate_Bathroom,Balconette,"distance to subway, m"
0,271271157,4,4,"500000.0 руб./ За месяц, Залог - 500000 руб., ...",Дизайнерский,0,Во двор,"Можно с детьми, Можно с животными",3.0,4,1,Да,200.0,Монолитный,16,5,1,0,0,630
1,271634126,4,4,"500000.0 руб./ За месяц, Залог - 500000 руб., ...",Дизайнерский,0,На улицу и двор,Можно с детьми,3.5,1,1,Нет,198.0,Монолитно-кирпичный,16,5,2,1,0,560
2,271173086,4,4,"500000.0 руб./ За месяц, Залог - 500000 руб., ...",Евроремонт,0,На улицу и двор,Можно с детьми,3.2,1,0,Нет,200.0,Неизвестно,16,5,3,0,0,490


In [243]:
#обратно (нам не надо, но работает)
#le = le.fit(df['Parking'])
#df['Parking'] = le.inverse_transform(df_encode['Parking'])
#df

In [244]:
# Кодирование понятно чего
le = LabelEncoder()
le.fit(df['Renovation'])
df_encode['Renovation'] = le.transform(df_encode['Renovation'])

In [245]:
# Кодирование мусоропровода
le = LabelEncoder()
le.fit(df['Garbage_Chute'])
df_encode['Garbage_Chute'] = le.transform(df_encode['Garbage_Chute'])

In [246]:
#животные/дети
df['Children/Pets_Allowed'].value_counts()
#pd.get_dummies(df_encode, columns=['Children/Pets_Allowed'])

Children/Pets_Allowed
Можно с детьми                       8540
Можно с детьми, Можно с животными    6075
по договорённости                    4915
Можно с животными                     207
Name: count, dtype: int64

In [247]:
# кодирование House_Type
le = LabelEncoder()
le.fit(df['House_Type'])
df_encode['House_Type'] = le.transform(df_encode['House_Type'])

In [248]:
# кодирование Windows
le = LabelEncoder()
le.fit(df['Windows'])
df_encode['Windows'] = le.transform(df_encode['Windows'])

In [249]:
#str to int
df_encode['Number_of_Rooms'] = df_encode['Number_of_Rooms'].astype(int)
df_encode['Area_sq_m'] = df_encode['Area_sq_m'].astype(float)
df_encode['Children/Pets_Allowed'].value_counts()

Children/Pets_Allowed
Можно с детьми                       8540
Можно с детьми, Можно с животными    6075
по договорённости                    4915
Можно с животными                     207
Name: count, dtype: int64

In [250]:
# Разбиение одной колонки на две
df_encode['Childrens_allowed'] = df_encode['Children/Pets_Allowed'].str.contains('Можно с детьми', case=False)
df_encode['Pets_allowed'] = df_encode['Children/Pets_Allowed'].str.contains('Можно с животными', case=False)

In [251]:
# выбрасываем то, что разбивали
df_encode.drop('Children/Pets_Allowed', axis=1, inplace=True)

In [253]:
#str to int
df_encode['Childrens_allowed'] = df_encode['Childrens_allowed'].astype(int)
df_encode['Pets_allowed'] = df_encode['Pets_allowed'].astype(int)
df_encode.info()

<class 'pandas.core.frame.DataFrame'>
Index: 19737 entries, 0 to 23367
Data columns (total 21 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   Ad_ID                   19737 non-null  int64  
 1   Number_of_Rooms         19737 non-null  int32  
 2   Parking                 19737 non-null  int32  
 3   Price                   19737 non-null  object 
 4   Renovation              19737 non-null  int32  
 5   Balcony                 19737 non-null  int64  
 6   Windows                 19737 non-null  int32  
 7   Ceiling_Height_M        19737 non-null  float64
 8   Count_pass_elevator     19737 non-null  int64  
 9   Count_freight_elevator  19737 non-null  int64  
 10  Garbage_Chute           19737 non-null  int32  
 11  Area_sq_m               19737 non-null  float64
 12  House_Type              19737 non-null  int32  
 13  Number_of_Floors        19737 non-null  int64  
 14  Floor_of_Apartment      19737 non-null  int